# Pre-requisites
###### Download your force data .kml file from: https://data.police.uk/data/boundaries/ and import to your data lake files.

*** DON'T forget to update the path to your force region ***

In [8]:
!pip install shapely

StatementMeta(, 3179709a-6fb2-49b0-a7ad-2e8686bc400f, 10, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.6 MB/s eta 0:00:00a 0:00:01


In [20]:
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Polygon
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, BooleanType
from pyspark.sql.functions import col, count

StatementMeta(, 3179709a-6fb2-49b0-a7ad-2e8686bc400f, 22, Finished, Available, Finished)

In [10]:
kml_path = "/lakehouse/default/Files/force kmls/cumbria.kml"
date = "2023-04"

StatementMeta(, 3179709a-6fb2-49b0-a7ad-2e8686bc400f, 12, Finished, Available, Finished)

In [11]:
with open(kml_path, 'r', encoding='utf-8') as file:
    kml_content = file.read()

# Step 2: Parse coordinates directly into list of (lon, lat)
soup = BeautifulSoup(kml_content, 'xml')
placemarks = soup.find_all('Placemark')

coords = []
for placemark in placemarks:
    coords_tag = placemark.find('coordinates')
    if coords_tag:
        raw_coords = coords_tag.text.strip().split()
        for coord in raw_coords:
            try:
                lon, lat = map(float, coord.split(',')[:2])
                coords.append((lon, lat))
            except ValueError:
                continue

# Step 3: Build polygon and simplify
polygon = Polygon(coords)
simplified = polygon.simplify(0.04, preserve_topology=True)

# Step 4: Create API string in [lat,lng:lat,lng:...] format
polygon_str = ":".join(f"{lat},{lon}" for lon, lat in simplified.exterior.coords)

# Step 5: Use in API
url = f"https://data.police.uk/api/crimes-street/all-crime?poly={polygon_str}&date={date}"

StatementMeta(, 3179709a-6fb2-49b0-a7ad-2e8686bc400f, 13, Finished, Available, Finished)

In [21]:
# Fetch data
response = requests.get(url)
response.raise_for_status()
data = response.json()

# Define nested schema
schema = StructType([
    StructField("category", StringType(), True),
    StructField("persistent_id", StringType(), True),
    StructField("location_subtype", StringType(), True),
    StructField("id", StringType(), True),
    StructField("location_type", StringType(), True),
    StructField("context", StringType(), True),
    StructField("month", StringType(), True),
    StructField("location", StructType([
        StructField("latitude", StringType(), True),
        StructField("longitude", StringType(), True),
        StructField("street", StructType([
            StructField("id", StringType(), True),
            StructField("name", StringType(), True)
        ]))
    ])),
    StructField("outcome_status", StructType([
        StructField("category", StringType(), True),
        StructField("date", StringType(), True)
    ]))
])

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

# Select nested fields and flatten for display
df_flat = df.select(
    "category",
    "month",
    "location.latitude",
    "location.longitude",
    "location.street.name",
    col("outcome_status.category").alias("outcome_category"),
    col("outcome_status.date").alias("outcome_date")
)

df_flat.createOrReplaceTempView("crimes")
spark.sql("SELECT category, COUNT(*) AS count FROM crimes GROUP BY category").show()

StatementMeta(, 3179709a-6fb2-49b0-a7ad-2e8686bc400f, 23, Finished, Available, Finished)

+--------------------+-----+
|            category|count|
+--------------------+-----+
|anti-social-behav...|  374|
|            burglary|  102|
|       bicycle-theft|   21|
|criminal-damage-a...|  478|
|         other-theft|  202|
|               drugs|  106|
|possession-of-wea...|   45|
|        public-order|  360|
|             robbery|   14|
|         shoplifting|  127|
|       vehicle-crime|   87|
|theft-from-the-pe...|    8|
|       violent-crime| 1434|
|         other-crime|   46|
+--------------------+-----+



In [22]:
query = """
            Select 
                category
                ,month
                ,cast(latitude as float) as latitude
                ,cast(longitude as float) as longitude
                ,name
                ,outcome_category
                ,outcome_date
            from crimes
        """

df_crimes = spark.sql(query)

StatementMeta(, 3179709a-6fb2-49b0-a7ad-2e8686bc400f, 24, Finished, Available, Finished)

In [23]:
## save the crime data to table

df_crimes.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("force_crime_data")

StatementMeta(, 3179709a-6fb2-49b0-a7ad-2e8686bc400f, 25, Finished, Available, Finished)